In [1]:
import datetime
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
import ribasim.nodes

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import *
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
waterschap = "Delfland"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = -0.42  # default LevelBoundary level

## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_0"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed
)
processor.run()

Swapped edge direction between Node A: 18 and Node B: 377
Swapped edge direction between Node A: 377 and Node B: 57
Swapped edge direction between Node A: 27 and Node B: 298
Swapped edge direction between Node A: 298 and Node B: 114
Processed all actions
The function of the pumps have been updated.


#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

# Parameterization

## Nodes

### Basin (characteristics)

In [5]:
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


In [6]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [7]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [8]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

#change high initial states to 0
ribasim_model.basin.state.df.loc[ribasim_model.basin.state.df["level"] == 9.999, "level"] = 0
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == 9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)


In [9]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

In [10]:
# #Add meta_node_id
# ribasim_model.basin.profile.df['meta_node_id'] = ribasim_model.basin.profile.df['node_id']
# ribasim_model.basin.area.df['meta_node_id'] = ribasim_model.basin.area.df['node_id']
# ribasim_model.basin.node.df['meta_node_id'] = ribasim_model.basin.node.df.index

# ribasim_model.level_boundary.node.df['meta_node_id'] = ribasim_model.level_boundary.node.df.index
# ribasim_model.flow_boundary.node.df['meta_node_id'] = ribasim_model.flow_boundary.node.df.index
# ribasim_model.pump.node.df['meta_node_id'] = ribasim_model.pump.node.df.index


## Implement standard profile and a storage basin

In [11]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [12]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Basin (forcing)

In [13]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10*2),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [14]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [15]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

### Add Outlet

In [16]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [17]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [18]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [19]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [20]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


### Manning Resistance

In [21]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [22]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 100
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [23]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)



Following node_ids are not connected to any edges:
               Type  node_id
729  LevelBoundary      549


# Set numerical settings

In [24]:
ribasim_model.use_validation = True

In [25]:
# Write model output
# ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2025, 1, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

## Iterate over tabulated rating curves

In [26]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [27]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

## Iterate over tabulated rating curves

In [28]:
# try:
#     ribasim_param.iterate_TRC(
#         ribasim_param=ribasim_param,
#         allowed_tolerance=0.02,
#         max_iter=1,
#         expected_difference=0.1,
#         max_adjustment=0.25,
#         cmd=["ribasim", path_ribasim_toml],
#         output_dir=output_dir,
#         path_ribasim_toml=path_ribasim_toml,
#     )

# except Exception:
#     logging.error("The model was not able to run. Log file:")
#     log_file_path = os.path.join(output_dir, "ribasim.log")  # Update with the correct path to your log file
#     try:
#         with open(log_file_path) as log_file:
#             log_content = log_file.read()
#             print(log_content)
#     except Exception as log_exception:
#         logging.error(f"Could not read the log file: {log_exception}")

# Write model

In [29]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)

../../../../../Ribasim_networks/Waterschappen/Delfland/modellen/Delfland_parametrized/results
The model of waterboard Delfland has been uploaded to the goodcloud in the directory of boezemmodel!


## Open Output

In [30]:
df_basin = pd.read_feather(os.path.join(output_dir, "basin.arrow"))
df_basin

,time,node_id,storage,level,inflow_rate,outflow_rate,storage_rate,precipitation,evaporation,drainage,infiltration,balance_error,relative_error
0,2024-01-01 00:00:00,1,2.195916e+07,-0.430000,1.241118e-01,0.044768,2.627284,2.547940,0.0,0.0,0.0,4.440892e-16,3.269185e-16
1,2024-01-01 00:00:00,2,2.849461e+06,-0.430000,5.702251e-01,0.387104,0.513746,0.330625,0.0,0.0,0.0,-1.110223e-16,-1.724009e-16
2,2024-01-01 00:00:00,3,4.247546e+04,-2.660000,2.585266e-01,0.259783,0.003672,0.004928,0.0,0.0,0.0,3.469447e-17,1.326145e-16
3,2024-01-01 00:00:00,4,5.722074e+05,-3.270000,2.041708e-02,0.052920,0.033891,0.066394,0.0,0.0,0.0,0.000000e+00,0.000000e+00
4,2024-01-01 00:00:00,5,3.935989e+05,-1.800000,1.198974e-02,0.027256,0.030404,0.045670,0.0,0.0,0.0,3.469447e-18,8.171576e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906123,2024-12-31 23:00:00,672,4.519021e+05,-1.911687,0.000000e+00,0.052217,0.000000,0.052217,0.0,0.0,0.0,9.700574e-15,1.857743e-13
1906124,2024-12-31 23:00:00,673,4.086998e+05,-2.340892,0.000000e+00,0.047206,0.000000,0.047206,0.0,0.0,0.0,-2.643025e-14,-5.598886e-13
1906125,2024-12-31 23:00:00,674,2.863590e+05,7.458248,6.684285e-09,0.000000,0.006903,0.006903,0.0,0.0,0.0,-2.898723e-15,-8.398996e-13
1906126,2024-12-31 23:00:00,675,7.807867e+04,-2.415476,0.000000e+00,0.009039,0.000000,0.009039,0.0,0.0,0.0,-2.746067e-15,-3.038010e-13


### Add discrete control nodes

In [31]:
# Add discrete control nodes and control edges
# ribasim_param.add_discrete_control_nodes(ribasim_model)

